<a href="https://colab.research.google.com/github/amanglarus/image_classification_using-tensorflow/blob/main/classification_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
!cp /content/drive/MyDrive/tshirt_laptop_data.zip /content/

In [ ]:
!unzip tshirt_laptop_data.zip

In [19]:
#!rm -r tshirt_laptop_data

## Convert all images to gray scale images

In [19]:
def rgb_to_gray(rgb_img): # provide image in rgb order not in bgr or any other order

  gamma = 1.04
  r_const, g_const, b_const = 0.2126, 0.7152, 0.0722

  r, g, b = rgb_img[:,:,0], rgb_img[:,:,1], rgb_img[:,:,2]

  gray_img = r_const*r**gamma + g_const*g**gamma + b_const*b**gamma

  return gray_img

In [20]:
grey_img = True

import glob
import cv2
dir_name = "/content/tshirt_laptop_data"+"/*"
if grey_img:
    for folder_name in glob.glob(dir_name):
      for data_type_path in glob.glob(folder_name+"/*"):
          for img_path in glob.glob(data_type_path+"/*"):
              
              img = cv2.imread(img_path) #BGR color map
              rgb_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # BGR to RGB
              gray = rgb_to_gray(rgb_img)
              cv2.imwrite(img_path, gray)


# Install specific tensorflow version 

In [22]:
!pip install tensorflow-gpu==2.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 75.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 64.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 KB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.7 MB/s eta 0:00:00
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=6d632e84e64b06473cbec679a8f3b72a5cbaeb45317b00203e407fc9d3d2643f
  Stored in directory: /root/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0

In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.0


# Load data in pipe line to tarin the model

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential

In [3]:
NUM_CLASSES = 2
BATCH_SIZE=32
IMAGE_SIZE = 96
NUM_CHANNEL = 1

IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
MODEL_IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNEL)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir='/content/tshirt_laptop_data/train'
val_data_dir='/content/tshirt_laptop_data/test'

# from tensorflow.keras.layers.preprocessing.image_preprocessing import HORIZONTAL

train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=10,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 horizontal_flip=True,
                                 fill_mode='nearest')

val_datagen=ImageDataGenerator(rescale=1./255)




train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=IMAGE_SHAPE,
                                                  batch_size=BATCH_SIZE,
                                                  color_mode="grayscale",
                                                  class_mode='sparse')

val_generator=val_datagen.flow_from_directory(val_data_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              color_mode="grayscale",
                                              class_mode='sparse')






Found 2385 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


# Build Model

### Mobilenet V3 from scratch


In [9]:
# code taken from here : https://github.com/sirius-ai/MobileNetV3-TF/blob/master/MobileNetV3.pyhttps://github.com/sirius-ai/MobileNetV3-TF/blob/master/MobileNetV3.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf


MobileNetV3_Small_Spec = [
    # Op            k    exp    out    SE     NL        s
    [ "ConvBnAct",  3,   False, 16,    False, "hswish", 2 ],
    [ "bneck",      3,   16,    16,    True,  "relu",   2 ],
    [ "bneck",      3,   72,    24,    False, "relu",   2 ],
    [ "bneck",      3,   88,    24,    False, "relu",   1 ],
    [ "bneck",      5,   96,    40,    True,  "hswish", 2 ],
    [ "bneck",      5,   240,   40,    True,  "hswish", 1 ],
    [ "bneck",      5,   240,   40,    True,  "hswish", 1 ],
    [ "bneck",      5,   120,   48,    True,  "hswish", 1 ],
    [ "bneck",      5,   144,   48,    True,  "hswish", 1 ],
    [ "bneck",      5,   288,   96,    True,  "hswish", 2 ],
    [ "bneck",      5,   576,   96,    True,  "hswish", 1 ],
    [ "bneck",      5,   576,   96,    True,  "hswish", 1 ],
    [ "ConvBnAct",  1,   False, 576,   True,  "hswish", 1 ],
    [ "pool",       7,   False, False, False, "None",   1 ],
    [ "ConvNBnAct", 1,   False, 1280,  False, "hswish", 1 ],
    [ "ConvNBnAct", 1,   False, 1000,  False, "None",   1 ],
]


MobileNetV3_Large_Spec = [
    # Op            k    exp    out    SE     NL        s
    [ "ConvBnAct",  3,   False, 16,    False, "hswish", 2 ],
    [ "bneck",      3,   16,    16,    False, "relu",   1 ],
    [ "bneck",      3,   64,    24,    False, "relu",   2 ],
    [ "bneck",      3,   72,    24,    False, "relu",   1 ],
    [ "bneck",      5,   72,    40,    True,  "relu",   2 ],
    [ "bneck",      5,   120,   40,    True,  "relu",   1 ],
    [ "bneck",      5,   120,   40,    True,  "relu",   1 ],
    [ "bneck",      3,   240,   80,    False, "hswish", 2 ],
    [ "bneck",      3,   200,   80,    False, "hswish", 1 ],
    [ "bneck",      3,   184,   80,    False, "hswish", 1 ],
    [ "bneck",      3,   184,   80,    False, "hswish", 1 ],
    [ "bneck",      3,   480,   112,   True,  "hswish", 1 ],
    [ "bneck",      3,   672,   112,   True,  "hswish", 1 ],
    [ "bneck",      5,   672,   160,   True,  "hswish", 2 ],
    [ "bneck",      5,   960,   160,   True,  "hswish", 1 ],
    [ "bneck",      5,   960,   160,   True,  "hswish", 1 ],
    [ "ConvBnAct",  1,   False, 960,   False, "hswish", 1 ],
    [ "pool",       7,   False, False, False, "None",   1 ],
    [ "ConvNBnAct", 1,   False, 1280,  False, "hswish", 1 ],
    [ "ConvNBnAct", 1,   False, 1000,  False, "None",   1 ],
]

def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)

    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor

    return new_v

class Identity(tf.keras.layers.Layer):
    def __init__(self, name="Identity", **kwargs):
        super(Identity, self).__init__(name=name, **kwargs)

    def call(self, input):
        return input

    def get_config(self):
        base_config = super(Identity, self).get_config()
        return dict(list(base_config.items()))

class HardSigmoid(tf.keras.layers.Layer):
    def __init__(self, name="HardSigmoid", **kwargs):
        super(HardSigmoid, self).__init__(name=name, **kwargs)
        self.relu6 = tf.keras.layers.ReLU(max_value=6, name="ReLU6", **kwargs)

    def call(self, input):
        return self.relu6(input + 3.0) / 6.0

    def get_config(self):
        base_config = super(HardSigmoid, self).get_config()
        return dict(list(base_config.items()))

class HardSwish(tf.keras.layers.Layer):
    def __init__(self, name="HardSwish", **kwargs):
        super(HardSwish, self).__init__(name=name, **kwargs)
        self.relu6 = tf.keras.layers.ReLU(max_value=6, name="ReLU6", **kwargs)

    def call(self, input):
        return input * self.relu6(input + 3.0) / 6.0

    def get_config(self):
        base_config = super(HardSwish, self).get_config()
        return dict(list(base_config.items()))

_available_activation = {
            "relu": tf.keras.layers.ReLU(name="ReLU"),
            "relu6": tf.keras.layers.ReLU(max_value=6, name="ReLU6"),
            "hswish": HardSwish(),
            "hsigmoid": HardSigmoid(),
            "softmax": tf.keras.layers.Softmax(name="Softmax"),
            "None": Identity(),
        }

class SENet(tf.keras.layers.Layer):
    def __init__(self, reduction=4, l2=2e-4, name="SENet", **kwargs):
        super(SENet, self).__init__(name=name, **kwargs)
        self.reduction = reduction
        self.l2 = l2

    def build(self, input_shape):
        _, h, w, c = input_shape
        self.gap = tf.keras.layers.GlobalAveragePooling2D(name=f'AvgPool{h}x{w}')
        self.fc1 = tf.keras.layers.Dense(units=c//self.reduction, activation="relu", use_bias=False,
                                         kernel_regularizer=tf.keras.regularizers.l2(self.l2), name="Squeeze")
        self.fc2 = tf.keras.layers.Dense(units=c, activation=HardSigmoid(), use_bias=False,
                                         kernel_regularizer=tf.keras.regularizers.l2(self.l2), name="Excite")
        self.reshape = tf.keras.layers.Reshape((1, 1, c), name=f'Reshape_None_1_1_{c}')

        super().build(input_shape)

    def call(self, input):
        output = self.gap(input)
        output = self.fc1(output)
        output = self.fc2(output)
        output = self.reshape(output)
        return input * output

    def get_config(self):
        config = {"reduction":self.reduction, "l2":self.l2}
        base_config = super(SENet, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class ConvBnAct(tf.keras.layers.Layer):
    def __init__(self, k, exp, out, SE, NL, s, l2, name="ConvBnAct", **kwargs):
        super(ConvBnAct, self).__init__(name=name, **kwargs)
        self.k = k
        self.exp = exp
        self.out = out
        self.se = SE
        self.nl = NL
        self.s = s
        self.l2 = l2
        self.conv2d = tf.keras.layers.Conv2D(filters=out, kernel_size=k, strides=s, activation=None, padding="same",
                                             kernel_regularizer=tf.keras.regularizers.l2(l2), name="conv2d", **kwargs)
        self.bn = tf.keras.layers.BatchNormalization(momentum=0.99, name="BatchNormalization", **kwargs)
        self.act = _available_activation[NL]

    def call(self, input):
        output = self.conv2d(input)
        output = self.bn(output)
        output = self.act(output)
        return output

    def get_config(self):
        config = {"k":self.k, "exp":self.exp, "out":self.out, "SE":self.se, "NL":self.nl, "s":self.s, "l2":self.l2}
        base_config = super(ConvBnAct, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class ConvNBnAct(tf.keras.layers.Layer):
    def __init__(self, k, exp, out, SE, NL, s, l2, name="ConvNBnAct", **kwargs):
        super(ConvNBnAct, self).__init__(name=name, **kwargs)
        self.k = k
        self.exp = exp
        self.out = out
        self.se = SE
        self.nl = NL
        self.s = s
        self.l2 = l2
        self.act = _available_activation[NL]
        self.fn = tf.keras.layers.Conv2D(filters=out, kernel_size=k, strides=s, activation=self.act, padding="same",
                                         kernel_regularizer=tf.keras.regularizers.l2(l2),name="conv2d", **kwargs)

    def call(self, input):
        output = self.fn(input)
        return output

    def get_config(self):
        config = {"k":self.k, "exp":self.exp, "out":self.out, "SE":self.se, "NL":self.nl, "s":self.s, "l2":self.l2}
        base_config = super(ConvNBnAct, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class Pool(tf.keras.layers.Layer):
    def __init__(self, k, exp, out, SE, NL, s, l2, name="Pool", **kwargs):
        super(Pool, self).__init__(name=name, **kwargs)
        self.k = k
        self.exp = exp
        self.out = out
        self.se = SE
        self.nl = NL
        self.s = s
        self.l2 = l2
        self.gap = tf.keras.layers.AveragePooling2D(pool_size=(k, k), strides=1, name=f'AvgPool{k}x{k}', **kwargs)

    def call(self, input):
        output = self.gap(input)
        return output

    def get_config(self):
        config = {"k":self.k, "exp":self.exp, "out":self.out, "SE":self.se, "NL":self.nl, "s":self.s, "l2":self.l2}
        base_config = super(Pool, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class BottleNeck(tf.keras.layers.Layer):
    def __init__(self, k, exp, out, SE, NL, s, l2, name="BottleNeck", **kwargs):
        super(BottleNeck, self).__init__(name=name, **kwargs)
        self.k = k
        self.exp = exp
        self.out = out
        self.se = SE
        self.nl = NL
        self.s = s
        self.l2 = l2
        self.expand = ConvBnAct(k=1, exp=exp, out=exp, SE=SE, NL=NL, s=1, l2=l2, name="BottleNeckExpand", **kwargs)
        self.depthwise = tf.keras.layers.DepthwiseConv2D(
            kernel_size=k,
            strides=s,
            padding="same",
            use_bias=False,
            depthwise_regularizer=tf.keras.regularizers.l2(l2),
            name=f'Depthwise{k}x{k}',
            ** kwargs,
        )
        self.pointwise = tf.keras.layers.Conv2D(
            filters=out,
            kernel_size=1,
            strides=1,
            use_bias=False,
            kernel_regularizer=tf.keras.regularizers.l2(l2),
            name=f'Pointwise1x1',
            ** kwargs,
        )
        self.bn_1 = tf.keras.layers.BatchNormalization(momentum=0.99, name="BatchNormalization_1", **kwargs)
        self.bn_2 = tf.keras.layers.BatchNormalization(momentum=0.99, name="BatchNormalization_2", **kwargs)

        if self.se:
            self.SeNet = SENet(name="SEBottleneck", l2=l2, **kwargs)

        self.act = _available_activation[NL]

    def call(self, input):
        output = self.expand(input)
        output = self.depthwise(output)
        output = self.bn_1(output)
        if self.se:
            output = self.SeNet(output)
        output = self.act(output)
        output = self.pointwise(output)
        output = self.bn_2(output)

        if self.s == 1 and self.exp == self.out:
            return input + output
        else:
            return output

    def get_config(self):
        config = {"k":self.k, "exp":self.exp, "out":self.out, "SE":self.se, "NL":self.nl, "s":self.s, "l2":self.l2}
        base_config = super(BottleNeck, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

_available_mobilenetv3_spec = {
            "small": MobileNetV3_Small_Spec,
            "large": MobileNetV3_Large_Spec,
        }

_available_operation = {
            "ConvBnAct":  ConvBnAct,
            "bneck":      BottleNeck,
            "pool":       Pool,
            "ConvNBnAct": ConvNBnAct,
        }

class CusReshape(tf.keras.layers.Layer):
    def __init__(self, out, name="Reshape", **kwargs):
        super(CusReshape, self).__init__(name=name, **kwargs)
        self.out = out
        self.reshape = tf.keras.layers.Reshape((out,), name=f'Reshape_None_{out}', **kwargs)

    def call(self, input):
        output = self.reshape(input)
        return output

    def get_config(self):
        config = {"out":self.out}
        base_config = super(CusReshape, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class CusDropout(tf.keras.layers.Layer):
    def __init__(self, dropout_rate, name="Dropout", **kwargs):
        super(CusDropout, self).__init__(name=name, **kwargs)
        self.dropout_rate = dropout_rate
        self.dropout = tf.keras.layers.Dropout(rate=dropout_rate, name=f'Dropout', **kwargs)

    def call(self, input):
        output = self.dropout(input)
        return output

    def get_config(self):
        config = {"dropout_rate":self.dropout_rate}
        base_config = super(CusDropout, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def MobileNetV3(type="large", input_shape=(224, 224, 3), classes_number=2, width_multiplier=1.0,
                divisible_by=8, l2_reg=2e-5, dropout_rate=0.2, name="MobileNetV3"):
    spec = _available_mobilenetv3_spec[type]
    spec[-1][3] = classes_number  # bottlenet layer size or class numbers
    name = name + "_" + type

    inputs = tf.keras.layers.Input(shape=input_shape, name="inputs")

    for i, params in enumerate(spec):
        Op, k, exp, out, SE, NL, s = params
        inference_op = _available_operation[Op]

        if isinstance(exp, int):
            exp_ch = _make_divisible(exp * width_multiplier, divisible_by)
        else:
            exp_ch = None
        if isinstance(out, int):
            out_ch = _make_divisible(out * width_multiplier, divisible_by)
        else:
            out_ch = None
        if i == len(spec) - 1:  # fix output classes error.
            out_ch = classes_number

        op_name = f'{Op}_{i}'
        if i == 0:
            output = inference_op(k, exp_ch, out_ch, SE, NL, s, l2_reg, op_name)(inputs)
        else:
            output = inference_op(k, exp_ch, out_ch, SE, NL, s, l2_reg, op_name)(output)

        if (type == "small" and i == 14) or (type == "large" and i == 18):
            output = CusDropout(dropout_rate=dropout_rate)(output)

    outputs = CusReshape(out=classes_number)(output)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name=name)
    model.summary()

    return model

custom_objects = {
    "ConvBnAct" :  ConvBnAct,
    "BottleNeck":  BottleNeck,
    "Pool"      :  Pool,
    "ConvNBnAct":  ConvNBnAct,
    "CusReshape":  CusReshape,
    "CusDropout":  CusDropout,
}



for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.compat.v2.config.experimental.set_memory_growth(gpu, True)

model = MobileNetV3(type="small")

# model = MobileNetV3(type="large")

Model: "MobileNetV3_small"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 224, 224, 3)]     0         
                                                                 
 ConvBnAct_0 (ConvBnAct)     (None, 112, 112, 16)      512       
                                                                 
 bneck_1 (BottleNeck)        (None, 56, 56, 16)        992       
                                                                 
 bneck_2 (BottleNeck)        (None, 28, 28, 24)        4272      
                                                                 
 bneck_3 (BottleNeck)        (None, 28, 28, 24)        5904      
                                                                 
 bneck_4 (BottleNeck)        (None, 14, 14, 40)        14176     
                                                                 
 bneck_5 (BottleNeck)        (None, 14, 14, 40)  

In [ ]:
import tensorflow as tf

def conv_block(inputs, filters, kernel_size, strides, use_bias=True):
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=strides, padding='same', use_bias=use_bias)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x

def depthwise_conv_block(inputs, strides, use_bias=True):
    x = tf.keras.layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same', use_bias=use_bias)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

def bottleneck(inputs, filters, kernel_size, expansion_factor, strides, use_bias=True):
    in_channels = inputs.shape[-1]
    x = depthwise_conv_block(inputs, 1, use_bias)
    x = depthwise_conv_block(x, 1 use_bias)
    x = tf.keras.layers.Conv2D(filters, 1, strides=1, padding='same', use_bias=use_bias)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if strides == 1 and in_channels == filters:
        x = tf.keras.layers.Add()([x, inputs])
    return x

def mobilenetv3_small(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)


    # # x3 = tf.keras.layers.Conv2D(1, 3, strides=1, padding='valid', use_bias=True)(inputs)
    # # x3 = tf.keras.layers.BatchNormalization()(x3)
    # # x3 = tf.keras.layers.ReLU()(x3)
    x1 = tf.keras.layers.AveragePooling2D(pool_size=(5, 5), strides=3)(inputs)
    x2 = tf.keras.layers.MaxPooling2D(pool_size=(5, 5), strides=3)(inputs)
    x = tf.keras.layers.Concatenate()([x1, x2])

    x = conv_block(x, 4, 3, strides=2)
    x = bottleneck(x, 6, 3, expansion_factor=1, strides=2)
    x = bottleneck(x, 6, 5, expansion_factor=1, strides=1)
    x = bottleneck(x, 6, 5, expansion_factor=1, strides=1)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=2)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=1)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=1)
    # x = conv_block(x, 8, 1, strides=1)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(48)(x)
    x = tf.keras.layers.ReLU(6.)(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, x)

# example usage
model = mobilenetv3_small(input_shape=MODEL_IMAGE_SHAPE, num_classes=NUM_CLASSES)
model.summary()


In [69]:
import tensorflow as tf

def conv_block(inputs, filters, kernel_size, strides, use_bias=True):
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=strides, padding='same', use_bias=use_bias)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x

def depthwise_conv_block(inputs, strides, use_bias=True):
    x = tf.keras.layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same', use_bias=use_bias)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

def bottleneck(inputs, filters, kernel_size, expansion_factor, strides, use_bias=True):
    in_channels = inputs.shape[-1]
    x = conv_block(inputs, expansion_factor * in_channels, 1, 1, use_bias)
    x = conv_block(x, expansion_factor * in_channels, kernel_size, strides, use_bias)
    x = tf.keras.layers.Conv2D(filters, 1, strides=1, padding='same', use_bias=use_bias)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if strides == 1 and in_channels == filters:
        x = tf.keras.layers.Add()([x, inputs])
    return x

def mobilenetv3_small(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)

    x1 = tf.keras.layers.AveragePooling2D(pool_size=(5, 5), strides=3)(inputs)
    x2 = tf.keras.layers.MaxPooling2D(pool_size=(5, 5), strides=3)(inputs)
    # # x3 = tf.keras.layers.Conv2D(1, 3, strides=1, padding='valid', use_bias=True)(inputs)
    # # x3 = tf.keras.layers.BatchNormalization()(x3)
    # # x3 = tf.keras.layers.ReLU()(x3)
    x = tf.keras.layers.Concatenate()([x1, x2])

    x = conv_block(x, 4, 3, strides=2)
    x = bottleneck(x, 6, 3, expansion_factor=1, strides=2)
    x = bottleneck(x, 6, 5, expansion_factor=1, strides=1)
    x = bottleneck(x, 6, 5, expansion_factor=1, strides=1)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=2)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=1)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=1)
    # x = conv_block(x, 8, 1, strides=1)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(48)(x)
    x = tf.keras.layers.ReLU(6.)(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, x)

# example usage
model = mobilenetv3_small(input_shape=MODEL_IMAGE_SHAPE, num_classes=NUM_CLASSES)
model.summary()


Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_30 (InputLayer)           [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
average_pooling2d_13 (AveragePo (None, 31, 31, 1)    0           input_30[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_16 (MaxPooling2D) (None, 31, 31, 1)    0           input_30[0][0]                   
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 31, 31, 2)    0           average_pooling2d_13[0][0]       
                                                                 max_pooling2d_16[0][0]    

### Mobilenet V2 from scratch

In [ ]:
import tensorflow as tf

def conv_block(inputs, filters, kernel_size, strides):
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x

def bottleneck(inputs, filters, kernel_size, t, strides, use_bias=False):
    in_channels = inputs.shape[-1]
    x = conv_block(inputs, t * in_channels, 1, 1)
    x = tf.keras.layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same', use_bias=use_bias)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv2D(filters, 1, strides=1, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if strides == 1 and in_channels == filters:
        x = tf.keras.layers.Add()([x, inputs])
    return x

def mobilenetv2(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = conv_block(inputs, 32, 3, strides=2)
    x = bottleneck(x, 16, 3, t=1, strides=1)
    x = bottleneck(x, 24, 3, t=6, strides=2)
    x = bottleneck(x, 24, 3, t=6, strides=1)
    x = bottleneck(x, 32, 3, t=6, strides=2)
    x = bottleneck(x, 32, 3, t=6, strides=1)
    x = bottleneck(x, 64, 3, t=6, strides=2)
    x = bottleneck(x, 64, 3, t=6, strides=1)
    x = bottleneck(x, 96, 3, t=6, strides=1)
    x = bottleneck(x, 96, 3, t=6, strides=1)
    x = bottleneck(x, 160, 3, t=6, strides=2)
    x = bottleneck(x, 160, 3, t=6, strides=1)
    x = bottleneck(x, 320, 3, t=6, strides=1)
    x = conv_block(x, 1280, 1, strides=1)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, x)

# example usage
model = mobilenetv2(input_shape=(224, 224, 3), num_classes=10)
model.summary()


In [56]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications import MobileNetV3Large
#tf.keras.applications.MobileNetV3Small

base_model = MobileNetV3Large(
    input_shape=MODEL_IMAGE_SHAPE,
    alpha=0.4,
    include_top=False,
    weights=None, #  'imagenet'
    #input_tensor=None,
    #pooling=None,
    #classes=1000,
    #classifier_activation='softmax',
    #**kwargs
)

for layer in base_model.layers: #base_model.layers[:-6] means that consider layer only till 6th last layer from layer 1
    layer.trainable = True  



# penultimate_layer = model.layers[-2]
# new_top_layer = tf.keras.layers.Dense(1)(penultimate_layer.output)  # create new FC layer and connect it to the rest of the model
# new_model = tf.keras.models.Model(model.input, new_top_layer)



# Use a Sequential model to add a trainable classifier on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax"),
])

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
MobilenetV3large (Functional (None, 3, 3, 1280)        1031008   
_________________________________________________________________
global_average_pooling2d_17  (None, 1280)              0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_34 (Dense)             (None, 64)                81984     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 130       
Total params: 1,113,122
Trainable params: 1,102,962
Non-trainable params: 10,160
_______________________________________

##### MobileNetV2

In [22]:
# https://github.com/Haikoitoh/paper-implementation/blob/main/MobileNetV2.ipynb

from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, ReLU, BatchNormalization, add,Softmax, AveragePooling2D, Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model

def expansion_block(x,t,filters,block_id):
    prefix = 'block_{}_'.format(block_id)
    total_filters = t*filters
    x = Conv2D(total_filters,1,padding='same',use_bias=False, name = prefix +'expand')(x)
    x = BatchNormalization(name=prefix +'expand_bn')(x)
    x = ReLU(6,name = prefix +'expand_relu')(x)
    return x

def depthwise_block(x,stride,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = DepthwiseConv2D(3,strides=(stride,stride),padding ='same', use_bias = False, name = prefix + 'depthwise_conv')(x)
    x = BatchNormalization(name=prefix +'dw_bn')(x)
    x = ReLU(6,name=prefix +'dw_relu')(x)
    return x

def projection_block(x,out_channels,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = Conv2D(filters = out_channels,kernel_size = 1,padding='same',use_bias=False,name= prefix + 'compress')(x)
    x = BatchNormalization(name=prefix +'compress_bn')(x)
    return x


def Bottleneck(x,t,filters, out_channels,stride,block_id):
    y = expansion_block(x,t,filters,block_id)
    y = depthwise_block(y,stride,block_id)
    y = projection_block(y, out_channels,block_id)
    if y.shape[-1]==x.shape[-1]:
        y = add([x,y])
    return y


def MobileNetV2(input_image = (224,224,3), n_classes=1000):
    input = Input(input_image)

    x1 = tf.keras.layers.AveragePooling2D(pool_size=(5, 5), strides=3)(input)
    x2 = tf.keras.layers.MaxPooling2D(pool_size=(5, 5), strides=3)(input)
    x = tf.keras.layers.Concatenate()([x1, x2])    

    x = Conv2D(1,kernel_size=3,strides=(2,2),padding = 'same', use_bias=False)(x)
    x = BatchNormalization(name='conv1_bn')(x)
    x = ReLU(6, name = 'conv1_relu')(x)

    # 17 Bottlenecks

    x = depthwise_block(x,stride=1,block_id=1)
    x = projection_block(x, out_channels=4,block_id=1)

    x = Bottleneck(x, t = 1, filters = x.shape[-1], out_channels = 6, stride = 2,block_id = 2)
    x = Bottleneck(x, t = 1, filters = x.shape[-1], out_channels = 6, stride = 1,block_id = 3)

    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 2,block_id = 4)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 1,block_id = 5)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 1,block_id = 6)

    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 2,block_id = 7)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 8)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 9)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 10)

    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 11)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 12)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 13)

    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 2,block_id = 14)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 1,block_id = 15)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 1,block_id = 16)

    # x = Bottleneck(x, t = 2, filters = x.shape[-1], out_channels = 8, stride = 1,block_id = 17)


    #1*1 conv
    x = Conv2D(filters = 6,kernel_size = 1,padding='same',use_bias=False, name = 'last_conv')(x)
    x = BatchNormalization(name='last_bn')(x)
    x = ReLU(6,name='last_relu')(x)

    #AvgPool 7*7
    x = GlobalAveragePooling2D(name='global_average_pool')(x)
    x = tf.keras.layers.Dense(32, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    output = Dense(n_classes,activation='softmax')(x)

    model = Model(input, output)

    return model


# n_classes = 1000
# input_shape = (224,224,3)

# model = MobileNetV2(input_shape,n_classes)
# model.summary()   

model = MobileNetV2(MODEL_IMAGE_SHAPE, NUM_CLASSES)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 31, 31, 1)    0           input_6[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 31, 31, 1)    0           input_6[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 31, 31, 2)    0           average_pooling2d[0][0]          
                                                                 max_pooling2d[0][0]        

# Train the model

In [25]:
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("image_classification_checkpoint.h5",
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=2,
                          verbose=1,
                          restore_best_weights=True)

callbacks=[checkpoint,earlystop]

model.compile(loss='SparseCategoricalCrossentropy',
                   optimizer=Adam(learning_rate=0.0015),
                   metrics=['sparse_categorical_accuracy'])

epochs=200

batch_size = 32

history = model.fit(train_generator,
                        #  steps_per_epoch=num_train_samples//batch_size,
                         epochs=epochs,
                        #  callbacks=callbacks,
                         validation_data=val_generator,
                        #  validation_steps=num_val_samples//batch_size
                         )

Epoch 1/200
75/75 [==============================] - 17s 194ms/step - loss: 0.6100 - sparse_categorical_accuracy: 0.6728 - val_loss: 0.5774 - val_sparse_categorical_accuracy: 0.7200
Epoch 2/200
75/75 [==============================] - 13s 169ms/step - loss: 0.5992 - sparse_categorical_accuracy: 0.6952 - val_loss: 0.6846 - val_sparse_categorical_accuracy: 0.6300
Epoch 3/200
75/75 [==============================] - 11s 140ms/step - loss: 0.5957 - sparse_categorical_accuracy: 0.6826 - val_loss: 0.5928 - val_sparse_categorical_accuracy: 0.7050
Epoch 4/200
75/75 [==============================] - 13s 175ms/step - loss: 0.5705 - sparse_categorical_accuracy: 0.7134 - val_loss: 0.5707 - val_sparse_categorical_accuracy: 0.7150
Epoch 5/200
75/75 [==============================] - 13s 174ms/step - loss: 0.5975 - sparse_categorical_accuracy: 0.6851 - val_loss: 0.5442 - val_sparse_categorical_accuracy: 0.7250
Epoch 6/200
75/75 [==============================] - 11s 141ms/step - loss: 0.5894 - spars

KeyboardInterrupt: ignored

# Convert from tensorflow to tensorflow lite

In [7]:
datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=10,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 horizontal_flip=True,
                                 fill_mode='nearest')
generator=datagen.flow_from_directory(val_data_dir,
                                      target_size=IMAGE_SHAPE,
                                      batch_size=BATCH_SIZE,
                                      color_mode="grayscale",
                                      class_mode='sparse')
def representative_data_gen():
  i = 0
  for image_batch, labels_batch in generator:
    i = i+1
    if i > 20:
      break;
    yield [image_batch]

Found 200 images belonging to 2 classes.


In [18]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

print('\nSetting the optimization flags..')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

print('\nConverting..')
tflite_model = converter.convert()

open("person_detect_model_data.tflite", "wb").write(tflite_model)

print("Done Conversion")


Setting the optimization flags..

Converting..
Done Conversion


# Convert from tensorflow to c array

In [19]:
!xxd -i ./person_detect_model_data.tflite > person_detect_model_data.cpp

In [20]:
#!cp /content/person_detect_model_data.cpp /content/person_detect_model_data_copy.cpp

In [21]:

search_text1 = "unsigned char __person_detect_model_data_tflite[]"


replace_text1 = str('#include "person_detect_model_data.h" \n\
#ifdef __has_attribute \n\
#define HAVE_ATTRIBUTE(x) __has_attribute(x) \n\
#else \n\
#define HAVE_ATTRIBUTE(x) 0 \n\
#endif \n\
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__)) \n\
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4))) \n\
#else \n\
#define DATA_ALIGN_ATTRIBUTE \n\
#endif \n\
const unsigned char g_person_detect_model_data[] DATA_ALIGN_ATTRIBUTE')


search_text2 = "unsigned int __person_detect_model_data_tflite_len"

replace_text2 = "const int g_person_detect_model_data_len"

with open(r'/content/person_detect_model_data.cpp', 'r') as file:

    data = file.read()

    data = data.replace(search_text1, replace_text1)
    data = data.replace(search_text2, replace_text2)
  # data = data.replace(search_text2, replace_text2)

with open(r'/content/person_detect_model_data.cpp', 'w') as file:
    file.write(data)
    file.close()



print("Text replaced")


Text replaced
